In [ ]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
offers = pd.read_feather('offers.arrow')

In [ ]:
zipcodes = pd.read_csv('zipcodes.csv')
zipcodes.columns = zipcodes.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

zipcodes = zipcodes.dropna(subset=['pna'])
zipcodes = zipcodes.astype({'adress_2': 'string'})

zipcodes['adress_2'] = zipcodes['adress_2'].str.strip()
streetno_rows = zipcodes['adress_2'].str.contains('\d*(/|-)(\d*|dk)')
zipcodes = zipcodes[~streetno_rows]
streetno_rows = zipcodes['adress_2'].str.contains('^\d*(,|)$')
zipcodes = zipcodes[~streetno_rows]

zipcodes['city'] = zipcodes['city'].str[1:]
zipcodes['adress'] = zipcodes.apply(
    lambda row:
        row.adress.replace(row.city.lower(), '').strip(),
        axis=1
    )


zipcodes['county'] = zipcodes['adress'].str.split().str[-1]

zipcodes['muni'] = zipcodes['adress'].str.split(' ').str[:-1].str.join('_')
zipcodes['muni'] = zipcodes.apply(
    lambda row:
        row.muni.replace(row.adress_2.replace(' ', '_')+'_', '') if len(row.muni) > 0 else 
            row.city.strip().lower().replace(' ', '_').replace('(', '').replace(')', ''),
        axis=1
    )

zipcodes['fullpath'] = zipcodes.apply(
    lambda row:
        row.woj + '-' + row.county + '-' + row.muni,
        axis=1
    )

municipalities = pd.read_csv('municipalities.csv')
municipalities['county'] = municipalities['county'].str.lower()
municipalities['muni'] = municipalities['municipality'].str.lower().str.replace(' ', '_')
municipalities = municipalities.replace(regex=['\[a\]'], value='')
municipalities['fullpath'] = municipalities.apply(
    lambda row:
        row.voivodeship + '-' + row.county + '-' + row.muni,
        axis=1
    )

new_df = pd.merge(zipcodes, municipalities,  how='outer', left_on='fullpath', right_on='fullpath')

In [ ]:
counties = geopandas.read_file('counties/counties.shp')
counties = counties.to_crs(epsg=4326)

munis = geopandas.read_file('munis/munis.shp')
munis = munis.to_crs(epsg=4326)

In [ ]:
gdf = geopandas.GeoDataFrame(
    offers, geometry=geopandas.points_from_xy(offers.longitude, offers.latitude))

gdf.plot()

In [ ]:
fig, ax = plt.subplots()

counties.plot(ax=ax)

gdf.plot(ax=ax, color='red', markersize=5);

fig.suptitle('CARS!')
ax.set_xlabel('longitude')
ax.set_ylabel('latitude')

In [ ]:
# DONT OPEN, DEAD INSIDE

zipcodes = pd.read_csv('zipcodes.csv')
zipcodes.columns = zipcodes.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

zipcodes = zipcodes.dropna(subset=['pna'])
zipcodes = zipcodes.astype({'adress_2': 'string'})

zipcodes['adress_2'] = zipcodes['adress_2'].str.strip()
streetno_rows = zipcodes['adress_2'].str.contains('\d*(/|-)(\d*|dk)')
zipcodes = zipcodes[~streetno_rows]
streetno_rows = zipcodes['adress_2'].str.contains('^\d*(,|)$')
zipcodes = zipcodes[~streetno_rows]
streetno_rows = zipcodes['adress_2'].str.contains('^\d*, \d*$')
zipcodes = zipcodes[~streetno_rows]


zipcodes['county'] = zipcodes['adress'].str.split().str[-1]

def polishpost_unscrew(row):
    address = row.adress.split(' ')
    if len(address) == 1: return address
    elif len(address) >= 2 and address[-1] == address [-2]: return address[-1]
    elif len(address) >= 4 and (address[-1] == address [-3]) and (address[-2] == address [-4]): return (address[-2] + ' ' + address[-1])
    else: return address[-1]

zipcodes['county'] = zipcodes.apply(
    lambda row:
        polishpost_unscrew(row),
        axis=1
    )

zipcodes['muni'] = zipcodes['adress'].str.split(' ').str[:-1].str.join('_')
zipcodes = zipcodes.drop_duplicates(subset='pna', keep="last")

In [ ]:
zipcodes = pd.read_excel('pna.xlsx')
zipcodes.columns = zipcodes.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

zipcodes['voivodeship'] = zipcodes['województwo'].str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
zipcodes['county'] = zipcodes['powiat'].str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
zipcodes['muni'] = zipcodes['gmina'].str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [ ]:
pna_db = pd.DataFrame()

pna_db[['pna', 'voivodeship', 'county', 'muni']] = zipcodes[['kod', 'województwo', 'powiat', 'gmina']]

# Some counties change names, yay?! (this one did in 2020)
pna_db['county'] = pna_db['county'].str.replace('jeleniogórski', 'karkonoski', regex=False)

In [ ]:
pna_soup = pd.read_csv('zipcodes.csv')
pna_soup.columns = pna_soup.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
pna_soup['city'] = pna_soup['city'].str[1:]

pna_toexpand = zipcodes[zipcodes['komentarz'] == 'całe miasto jako 1 kształt']
pna_city = pd.DataFrame()

for index, row in pna_toexpand.iterrows():
    if pna_city.empty:
        pna_city = pna_soup[pna_soup['city'] == row.gmina]
    else:
        pna_city_temp = pna_soup[pna_soup['city'] == row.gmina]
        pna_city = pd.concat([pna_city, pna_city_temp])
        
pna_cities = pd.DataFrame()
pna_cities[['pna', 'voivodeship', 'county']] = pna_city[['pna', 'woj', 'city']]
pna_cities['muni'] = pna_cities['county']

In [ ]:
municipalities = pd.read_csv('municipalities.csv')
municipalities['county'] = municipalities['county'].str.lower()
municipalities['muni'] = municipalities['municipality'].str.lower().str.replace(' ', '_')
municipalities = municipalities.replace(regex=['\[a\]'], value='')

municipalities['muni'] = municipalities['muni'].str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
municipalities['county'] = municipalities['county'].str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

municipalities['fullpath'] = municipalities.apply(
    lambda row:
        row.voivodeship + '-' + row.county + '-' + row.muni,
        axis=1
    )

In [ ]:
pna_db = pd.concat([pna_db, pna_cities], axis=0, ignore_index=True)

pna_db['muni'] = pna_db['muni'].str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
pna_db['county'] = pna_db['county'].str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

pna_db['fullpath'] = pna_db.apply(
    lambda row:
        row.voivodeship + '-' + row.county + '-' + row.muni,
        axis=1
    )

In [ ]:
#pna_teryt = pna_db.merge(municipalities, on='fullpath', how='outer', suffixes=(None, '_'))
# there's a couple dozen of PNAs with no matches left, good enough to figure out in Excel with PNA search tool

#pna_teryt.to_excel('pna_teryt.xlsx')

# With help of Wikipedia and GUS I managed to fill this out
# Most issues stem from alternative ways of how county name can be derived via inflection

#pna_teryt = pd.read_excel('pna_teryt.xlsx')

In [ ]:
pna_teryt = pd.read_excel('pna_teryt.xlsx')
#pna_teryt = pna_teryt.drop(['municipality', 'muni_', 'voivodeship_', 'county_'], axis=1)

In [ ]:
from tabula.io import read_pdf
tables = read_pdf('spispna.pdf', pages='all', multiple_tables=True, stream=True)  # substitute your file name

In [ ]:
tables[5]

In [ ]:
zipcodes = pd.read_csv('pnadb.csv')

In [1]:
import pandas as pd

offers = pd.read_feather('offers.arrow')